In [1]:
from __future__ import division  

import numpy as np
import pandas as pd
from geopy.distance import vincenty
import json

In [2]:
# define a function to give the distance between two ( latitude, longtitude) points
def getdistance(start_location, next_location):
    start_ = (start_location[0], start_location[1])
    next_ = (next_location[0], next_location[1])
    return vincenty(start_, next_).miles

In [3]:
# read the dataset, using json.loads to read the data structure --- list.

taxi_trips = pd.read_csv('/home/rui/Documents/MLProject/data/modified-crab/after-resample.csv')
trip_trace = taxi_trips['trip_trace']
new_triptrace = []
for i in xrange(trip_trace.shape[0]):
    new_triptrace.append(json.loads(trip_trace.iloc[i]))

taxi_trips['trip_trace'] = new_triptrace

In [4]:
taxi_trips.head()

,taxi_id,time_stamp,week_day,trip_trace
0,enyenewl,1.212620e+09,2,"[[[37.78401, -122.40783], 1212618405.0], [[37...."
1,enyenewl,1.212620e+09,2,"[[[37.79243, -122.42614], 1212619843.0], [[37...."
2,enyenewl,1.212625e+09,3,"[[[37.78684, -122.3998], 1212621717.0], [[37.7..."
3,enyenewl,1.212626e+09,3,"[[[37.8085, -122.41004], 1212625046.0], [[37.8..."
4,enyenewl,1.212626e+09,3,"[[[37.7934, -122.39326], 1212625685.0], [[37.7..."


In [5]:
taxi_trips.shape

(453048, 4)

# main function

In [6]:
speed_limit = 65

In [7]:
abnormal_indicator = []

for row_index, row in taxi_trips.iterrows():
    single_trip = row[3]
    trip_length = len(single_trip)
    
    start_location = single_trip[0][0]
    start_time = single_trip[0][1]
    
    wrong_indicator = 0
    
    if trip_length >= 3:
        for i in range(1, trip_length):
            if i > 40:
                break
                
            next_location = single_trip[i][0]
            next_time = single_trip[i][1]

            distance = getdistance(start_location, next_location) # in miles
            time_spend = next_time - start_time # in seconds

            # average speed
            speed = distance / (time_spend / 3600) # miles per hour

            # speed limit
            if speed <= speed_limit: 
                start_location = next_location
                start_time = next_time
            else:
                wrong_indicator = 1
                break
    elif trip_length < 3:
        wrong_indicator = 1
    abnormal_indicator.append(wrong_indicator)

taxi_trips['abnormal_indicator'] = abnormal_indicator
taxi_pandas = taxi_trips[taxi_trips['abnormal_indicator'] == 0]
del taxi_pandas['abnormal_indicator']
taxi_pandas.to_csv('/home/rui/Documents/MLProject/data/modified-crab/valid-trips-limit'+str(speed_limit) + '.csv', index=None)

In [8]:
len(abnormal_indicator)

453048

In [9]:
taxi_trips['abnormal_indicator'] = abnormal_indicator

In [ ]:
taxi_pandas = taxi_trips[taxi_trips['abnormal_indicator'] == 0]

In [10]:
taxi_pandas.shape

(280668, 4)

In [ ]:
del taxi_pandas['abnormal_indicator']

In [ ]:
taxi_pandas.head()

In [ ]:
taxi_pandas.to_csv('/home/rui/Documents/MLProject/data/modified-crab/valid-trips-limit65.csv', index=None)